In [2]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns
from pyscf.scf.uhf import det_ovlp

In [13]:
# Wavefunction generation
from pyscf import lib, gto, scf
from pyscf import gto, mp, mcscf
def H2_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="H 0. 0. 0.0; H 0. 0. 1.4",
        basis=f"ccecpccpvdz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.UHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, 3,2)
    # mc.fcisolver.nroots = 4
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc
    
def HF_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="H 0. 0. 0.0; F 0. 0. 0.8",
        basis=f"ccecpccpvdz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.UHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, 6,2)
    # mc.fcisolver.nroots = 4
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc

In [14]:
scf_checkfile = 'rohf.chk'
ci_checkfile = 'casci.chk'
for fname in [scf_checkfile, ci_checkfile]:
    if os.path.isfile(fname):
        os.remove(fname)
mol, mf, mcc = H2_casci(scf_checkfile, ci_checkfile)
# mol, mf, mcc = HF_casci(scf_checkfile, ci_checkfile)

dict_keys(['mol', '_scf', 'verbose', 'stdout', 'max_memory', 'ncas', 'nelecas', '_ncore', 'fcisolver', 'frozen', 'extrasym', 'e_tot', 'e_cas', 'ci', 'mo_coeff', 'mo_energy', 'mo_occ', 'converged'])


In [15]:
import bosonslater 
import pyscftools, mc
boson_wf = bosonslater.BosonWF(mol, mf, mc=mcc)

nconfig = 1000
mol, mf = pyscftools.recover_pyscf(scf_checkfile)
configs = mc.initial_guess(mol, nconfig)
# wf.parameters['det_coeff'] = np.ones(len(wf.parameters['det_coeff']))
signb, phib = boson_wf.recompute(configs)
num_ao = boson_wf._aovals.shape[-1]
print(num_ao)

10


In [22]:
mf.get_ovlp()

array([[ 1.        ,  0.92555844,  0.        ,  0.        ,  0.        ,
         0.7529535 ,  0.78435096,  0.        ,  0.        , -0.3941519 ],
       [ 0.92555844,  1.        ,  0.        ,  0.        ,  0.        ,
         0.78435096,  0.87263951,  0.        ,  0.        , -0.1887443 ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.48412712,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.48412712, -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.3941519 ,  0.1887443 ,  0.        ,  0.        , -0.21825202],
       [ 0.7529535 ,  0.78435096,  0.        ,  0.        ,  0.3941519 ,
         1.        ,  0.92555844,  0.        ,  0.        ,  0.        ],
       [ 0.78435096,  0.87263951,  0.        ,  0.        ,  0.1887443 ,
         0.92555844,  1.        ,  0.        

In [19]:
mf.mo_coeff.shape
# nup = np.einsum('ni, ni, nj, nj->nij', updet_sign, np.exp(updet_val), updet_sign, np.exp(updet_val)) # ui * uj

(2, 10, 10)

In [30]:
np.round(mf.get_ovlp() - np.einsum('ij,jk->ik', np.linalg.inv(mf.mo_coeff[0]).T, np.linalg.inv(mf.mo_coeff[0])), 6) # ui * uj

array([[ 0., -0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.],
       [-0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0., -0.,  0.,  0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.],
       [-0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.],
       [ 0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.]])